<a href="https://colab.research.google.com/github/ndb796/Deep-Learning-Paper-Review-and-Practice/blob/master/code_practices/ResNet18_CIFAR10_Train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch

torch.cuda.is_available()

True

In [2]:
import torch
import random
import torch.backends.cudnn as cudnn

torch.manual_seed(0)
torch.cuda.manual_seed(0)
torch.cuda.manual_seed_all(0)
cudnn.benchmark = False
cudnn.deterministic = True
random.seed(0)

#### ResNet18 모델 정의 및 인스턴스 초기화

In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.backends.cudnn as cudnn
import torch.optim as optim
import os


# ResNet18을 위해 최대한 간단히 수정한 BasicBlock 클래스 정의
class BasicBlock(nn.Module):
    def __init__(self, in_planes, planes, stride=1):
        super(BasicBlock, self).__init__()

        # 3x3 필터를 사용 (너비와 높이를 줄일 때는 stride 값 조절)
        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes) # 배치 정규화(batch normalization)

        # 3x3 필터를 사용 (패딩을 1만큼 주기 때문에 너비와 높이가 동일)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes) # 배치 정규화(batch normalization)

        self.shortcut = nn.Sequential() # identity인 경우
        if stride != 1: # stride가 1이 아니라면, Identity mapping이 아닌 경우
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, planes, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(planes)
            )

    def forward(self, x):
        out = F.leaky_relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x) # (핵심) skip connection
        out = F.leaky_relu(out)
        return out


# ResNet 클래스 정의
class ResNet(nn.Module):
    def __init__(self, block, num_blocks, num_classes=10):
        super(ResNet, self).__init__()
        self.in_planes = 64

        # 64개의 3x3 필터(filter)를 사용
        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.layer1 = self._make_layer(block, 64, num_blocks[0], stride=1)
        self.layer2 = self._make_layer(block, 128, num_blocks[1], stride=2)
        self.layer3 = self._make_layer(block, 256, num_blocks[2], stride=2)
        self.layer4 = self._make_layer(block, 512, num_blocks[3], stride=2)
        self.linear = nn.Linear(512, num_classes)

    def _make_layer(self, block, planes, num_blocks, stride):
        strides = [stride] + [1] * (num_blocks - 1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_planes, planes, stride))
            self.in_planes = planes # 다음 레이어를 위해 채널 수 변경
        return nn.Sequential(*layers)

    def forward(self, x):
        out = F.leaky_relu(self.bn1(self.conv1(x)))
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = F.avg_pool2d(out, 4)
        out = out.view(out.size(0), -1)
        out = self.linear(out)
        return out


# ResNet18 함수 정의
def ResNet18():
    return ResNet(BasicBlock, [2, 2, 2, 2])

#### 데이터셋(Dataset) 다운로드 및 불러오기

In [4]:
import torchvision
import torchvision.transforms as transforms

transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
])

train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform_train)
test_dataset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform_test)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=128, shuffle=True, num_workers=4)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=100, shuffle=False, num_workers=4)

Files already downloaded and verified
Files already downloaded and verified


#### 환경 설정 및 학습(Training) 함수 정의

In [5]:
device = 'cuda'

net = ResNet18()
net = net.to(device)
net = torch.nn.DataParallel(net)
cudnn.benchmark = True

learning_rate = 0.1
file_name = 'resnet18_cifar10.pt'

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=learning_rate, momentum=0.9, weight_decay=0.0002)


def train(epoch):
    print('\n[ Train epoch: %d ]' % epoch)
    net.train()
    train_loss = 0
    correct = 0
    total = 0
    for batch_idx, (inputs, targets) in enumerate(train_loader):
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()

        benign_outputs = net(inputs)
        loss = criterion(benign_outputs, targets)
        loss.backward()

        optimizer.step()
        train_loss += loss.item()
        _, predicted = benign_outputs.max(1)

        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()
        
        if batch_idx % 100 == 0:
            print('\nCurrent batch:', str(batch_idx))
            print('Current benign train accuracy:', str(predicted.eq(targets).sum().item() / targets.size(0)))
            print('Current benign train loss:', loss.item())

    print('\nTotal benign train accuarcy:', 100. * correct / total)
    print('Total benign train loss:', train_loss)


def test(epoch):
    print('\n[ Test epoch: %d ]' % epoch)
    net.eval()
    loss = 0
    correct = 0
    total = 0

    for batch_idx, (inputs, targets) in enumerate(test_loader):
        inputs, targets = inputs.to(device), targets.to(device)
        total += targets.size(0)

        outputs = net(inputs)
        loss += criterion(outputs, targets).item()

        _, predicted = outputs.max(1)
        correct += predicted.eq(targets).sum().item()

    print('\nTest accuarcy:', 100. * correct / total)
    print('Test average loss:', loss / total)

    state = {
        'net': net.state_dict()
    }
    if not os.path.isdir('checkpoint'):
        os.mkdir('checkpoint')
    torch.save(state, './checkpoint/' + file_name)
    print('Model Saved!')


def adjust_learning_rate(optimizer, epoch):
    lr = learning_rate
    if epoch >= 50:
        lr = 0.01
    if epoch >= 90:
        lr = 0.001
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr

#### 학습(Training) 진행

* 대략 20번의 epoch 이후에도 85%가량의 test accuracy를 얻을 수 있습니다.

In [6]:
# for epoch in range(0, 200):
for epoch in range(0, 100):
    adjust_learning_rate(optimizer, epoch)
    train(epoch)
    test(epoch)


[ Train epoch: 0 ]

Current batch: 0
Current benign train accuracy: 0.1171875
Current benign train loss: 2.360734462738037

Current batch: 100
Current benign train accuracy: 0.2890625
Current benign train loss: 1.8526116609573364

Current batch: 200
Current benign train accuracy: 0.3671875
Current benign train loss: 1.698861002922058

Current batch: 300
Current benign train accuracy: 0.4140625
Current benign train loss: 1.653452754020691

Total benign train accuarcy: 34.818
Total benign train loss: 702.8111106157303

[ Test epoch: 0 ]

Test accuarcy: 33.74
Test average loss: 0.018948830389976502
Model Saved!

[ Train epoch: 1 ]

Current batch: 0
Current benign train accuracy: 0.390625
Current benign train loss: 1.6778322458267212

Current batch: 100
Current benign train accuracy: 0.484375
Current benign train loss: 1.359874963760376

Current batch: 200
Current benign train accuracy: 0.46875
Current benign train loss: 1.448974609375

Current batch: 300
Current benign train accuracy: 0.


Current batch: 200
Current benign train accuracy: 0.90625
Current benign train loss: 0.3420089781284332

Current batch: 300
Current benign train accuracy: 0.90625
Current benign train loss: 0.29225218296051025

Total benign train accuarcy: 87.138
Total benign train loss: 144.2019355893135

[ Test epoch: 13 ]

Test accuarcy: 78.76
Test average loss: 0.00678543541431427
Model Saved!

[ Train epoch: 14 ]

Current batch: 0
Current benign train accuracy: 0.8828125
Current benign train loss: 0.35333630442619324

Current batch: 100
Current benign train accuracy: 0.8828125
Current benign train loss: 0.3051249086856842

Current batch: 200
Current benign train accuracy: 0.8125
Current benign train loss: 0.5103084444999695

Current batch: 300
Current benign train accuracy: 0.8828125
Current benign train loss: 0.35029616951942444

Total benign train accuarcy: 87.818
Total benign train loss: 137.54573778808117

[ Test epoch: 14 ]

Test accuarcy: 84.81
Test average loss: 0.004455768352746963
Model 


Total benign train accuarcy: 91.254
Total benign train loss: 98.9031682536006

[ Test epoch: 26 ]

Test accuarcy: 84.17
Test average loss: 0.005042848786711693
Model Saved!

[ Train epoch: 27 ]

Current batch: 0
Current benign train accuracy: 0.90625
Current benign train loss: 0.23792944848537445

Current batch: 100
Current benign train accuracy: 0.8984375
Current benign train loss: 0.2966381311416626

Current batch: 200
Current benign train accuracy: 0.8828125
Current benign train loss: 0.32082146406173706

Current batch: 300
Current benign train accuracy: 0.9140625
Current benign train loss: 0.2486734241247177

Total benign train accuarcy: 91.388
Total benign train loss: 96.4013108164072

[ Test epoch: 27 ]

Test accuarcy: 82.62
Test average loss: 0.005703511795401573
Model Saved!

[ Train epoch: 28 ]

Current batch: 0
Current benign train accuracy: 0.921875
Current benign train loss: 0.2000238299369812

Current batch: 100
Current benign train accuracy: 0.953125
Current benign train


Current batch: 0
Current benign train accuracy: 0.953125
Current benign train loss: 0.17030617594718933

Current batch: 100
Current benign train accuracy: 0.9296875
Current benign train loss: 0.20805901288986206

Current batch: 200
Current benign train accuracy: 0.9765625
Current benign train loss: 0.0828760489821434

Current batch: 300
Current benign train accuracy: 0.9375
Current benign train loss: 0.18178990483283997

Total benign train accuarcy: 92.754
Total benign train loss: 81.99019161611795

[ Test epoch: 40 ]

Test accuarcy: 81.32
Test average loss: 0.006120524019002914
Model Saved!

[ Train epoch: 41 ]

Current batch: 0
Current benign train accuracy: 0.984375
Current benign train loss: 0.07995420694351196

Current batch: 100
Current benign train accuracy: 0.921875
Current benign train loss: 0.1765691041946411

Current batch: 200
Current benign train accuracy: 0.921875
Current benign train loss: 0.21167321503162384

Current batch: 300
Current benign train accuracy: 0.90625
Cu


Current batch: 200
Current benign train accuracy: 0.984375
Current benign train loss: 0.03728880360722542

Current batch: 300
Current benign train accuracy: 0.9921875
Current benign train loss: 0.027687549591064453

Total benign train accuarcy: 98.606
Total benign train loss: 17.17004844918847

[ Test epoch: 53 ]

Test accuarcy: 93.61
Test average loss: 0.0021412189677357672
Model Saved!

[ Train epoch: 54 ]

Current batch: 0
Current benign train accuracy: 1.0
Current benign train loss: 0.007186941336840391

Current batch: 100
Current benign train accuracy: 0.9921875
Current benign train loss: 0.029575461521744728

Current batch: 200
Current benign train accuracy: 1.0
Current benign train loss: 0.01285205502063036

Current batch: 300
Current benign train accuracy: 0.984375
Current benign train loss: 0.050480447709560394

Total benign train accuarcy: 98.848
Total benign train loss: 14.499834300018847

[ Test epoch: 54 ]

Test accuarcy: 93.66
Test average loss: 0.0021513879392296075
Mod


Total benign train accuarcy: 99.7
Total benign train loss: 4.47563697444275

[ Test epoch: 66 ]

Test accuarcy: 94.1
Test average loss: 0.0024167520966380833
Model Saved!

[ Train epoch: 67 ]

Current batch: 0
Current benign train accuracy: 1.0
Current benign train loss: 0.003752082586288452

Current batch: 100
Current benign train accuracy: 1.0
Current benign train loss: 0.007977468892931938

Current batch: 200
Current benign train accuracy: 1.0
Current benign train loss: 0.003322640433907509

Current batch: 300
Current benign train accuracy: 1.0
Current benign train loss: 0.006016877014189959

Total benign train accuarcy: 99.782
Total benign train loss: 3.5887399803614244

[ Test epoch: 67 ]

Test accuarcy: 93.99
Test average loss: 0.002473105436936021
Model Saved!

[ Train epoch: 68 ]

Current batch: 0
Current benign train accuracy: 0.9921875
Current benign train loss: 0.02832089737057686

Current batch: 100
Current benign train accuracy: 1.0
Current benign train loss: 0.0036553509


Current batch: 100
Current benign train accuracy: 1.0
Current benign train loss: 0.003765665926039219

Current batch: 200
Current benign train accuracy: 1.0
Current benign train loss: 0.0026506243739277124

Current batch: 300
Current benign train accuracy: 1.0
Current benign train loss: 0.002345945220440626

Total benign train accuarcy: 99.864
Total benign train loss: 2.349701276398264

[ Test epoch: 80 ]

Test accuarcy: 94.0
Test average loss: 0.0026067570824176074
Model Saved!

[ Train epoch: 81 ]

Current batch: 0
Current benign train accuracy: 0.9921875
Current benign train loss: 0.018269723281264305

Current batch: 100
Current benign train accuracy: 1.0
Current benign train loss: 0.005314929410815239

Current batch: 200
Current benign train accuracy: 1.0
Current benign train loss: 0.0014427767600864172

Current batch: 300
Current benign train accuracy: 1.0
Current benign train loss: 0.0016991461161524057

Total benign train accuarcy: 99.894
Total benign train loss: 1.956722040224


Total benign train accuarcy: 99.954
Total benign train loss: 1.0968333568889648

[ Test epoch: 93 ]

Test accuarcy: 94.29
Test average loss: 0.0025013531602919104
Model Saved!

[ Train epoch: 94 ]

Current batch: 0
Current benign train accuracy: 1.0
Current benign train loss: 0.0013524133246392012

Current batch: 100
Current benign train accuracy: 1.0
Current benign train loss: 0.0008089039474725723

Current batch: 200
Current benign train accuracy: 1.0
Current benign train loss: 0.0025012062396854162

Current batch: 300
Current benign train accuracy: 1.0
Current benign train loss: 0.000948967004660517

Total benign train accuarcy: 99.96
Total benign train loss: 1.1154602876049466

[ Test epoch: 94 ]

Test accuarcy: 94.22
Test average loss: 0.0025310039296746255
Model Saved!

[ Train epoch: 95 ]

Current batch: 0
Current benign train accuracy: 1.0
Current benign train loss: 0.002599269850179553

Current batch: 100
Current benign train accuracy: 1.0
Current benign train loss: 0.0064140